# **IMPORT LIBRARIES**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import re
from google.colab import files

# **LOAD DATA**

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
# IMPORT DATA SAPRIL
training_features = pd.read_csv('/content/drive/MyDrive/Find IT UGM/train_features.csv')
training_label = pd.read_csv('/content/drive/MyDrive/Find IT UGM/train_label.csv')
test_feature = pd.read_csv('/content/drive/MyDrive/Find IT UGM/test_feature.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **EDA**
Understanding the problem

In [ ]:
training_features

,facilities,rating,location
0,RestaurantBARSwimmingPools,7.8 Very GoodFrom 10 reviews,Stokol
1,intrnetRestaurantgym,5.6 GoodFrom 4 reviews,Machlessvile
2,restaurantgympoolBar,7.2 Very GoodFrom 38 reviews,Wanderland
3,BARRestaurant,7.3 Very GoodFrom 6 reviews,Uberlandia
4,InternetRestaurant,7.2 Very GoodFrom 30 reviews,Stokol
...,...,...,...
3061,barInternet,NaN,Andeman
3062,restaurantBarInternet,8.1 ExcellentFrom 4 reviews,Uberlandia
3063,Barrestaurantswimmingpools,6.7 Very GoodFrom 10 reviews,Willsmian
3064,Restaurant,NaN,Hallerson


In [ ]:
test_feature

,ID,facilities,rating,location
0,0,GymrestaurantbarInternetSwimmingPools,8.0 ExcellentFrom 1 reviews,Stokol
1,1,Poolrestaurantgyminternetbar,7.4 Very GoodFrom 22 reviews,Hallerson
2,2,BARSwimmingPoolsInternetgym,0.0 FairFrom 4 reviews,Hallerson
3,3,gymSwimmingPoolsBARintrnetRestaurant,6.8 Very GoodFrom 13 reviews,Andeman
4,4,gymRestaurantpoolbarintrnet,0.0 FairFrom 9 reviews,Hallerson
...,...,...,...,...
762,762,BARPoolRestaurantGym,8.1 ExcellentFrom 9 reviews,Andeman
763,763,intrnetGympoolrestaurantBar,8.0 ExcellentFrom 4 reviews,Wanderland
764,764,RestaurantgymintrnetBARswimmingpools,7.4 Very GoodFrom 19 reviews,Andeman
765,765,gymInternetBarpoolRestaurant,9.0 ExcellentFrom 17 reviews,Hallerson


In [ ]:
#Mengecek Misssing Value
training_features.info()
training_features.isnull().sum()
training_features.isna().sum().sort_values()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066 entries, 0 to 3065
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   facilities  2765 non-null   object
 1   rating      2429 non-null   object
 2   location    3066 non-null   object
dtypes: object(3)
memory usage: 72.0+ KB


location        0
facilities    301
rating        637
dtype: int64

In [ ]:
#Cek unique Values
unique_counts = training_features['facilities'].value_counts()
unique_counts = pd.DataFrame(unique_counts)
unique_counts

,facilities
Restaurant,101
BARrestaurant,67
restaurant,65
BARRestaurant,65
restaurantBAR,56
...,...
RestaurantBARInternetpool,1
restaurantinternetSwimmingPoolsBarGym,1
PoolgymRestaurant,1
barRestaurantpool,1


We `convert` the all facilities' values of training_features to lowercase and `extract` values of rating

In [ ]:
# Convert all values to lowercase
training_features['facilities'] = training_features['facilities'].str.lower()
# Print the updated DataFrame
training_features

,facilities,rating,location
0,restaurantbarswimmingpools,7.8 Very GoodFrom 10 reviews,Stokol
1,intrnetrestaurantgym,5.6 GoodFrom 4 reviews,Machlessvile
2,restaurantgympoolbar,7.2 Very GoodFrom 38 reviews,Wanderland
3,barrestaurant,7.3 Very GoodFrom 6 reviews,Uberlandia
4,internetrestaurant,7.2 Very GoodFrom 30 reviews,Stokol
...,...,...,...
3061,barinternet,NaN,Andeman
3062,restaurantbarinternet,8.1 ExcellentFrom 4 reviews,Uberlandia
3063,barrestaurantswimmingpools,6.7 Very GoodFrom 10 reviews,Willsmian
3064,restaurant,NaN,Hallerson


In [ ]:
#training_features.to_csv('training_features_new.csv', header=True, index=False)
#files.download('training_features_new.csv')

We `convert` the all facilities' values of test_feature to lowercase and `extract` values of rating

In [ ]:
# Convert all values to lowercase
test_feature['facilities'] = test_feature['facilities'].str.lower()

# Print the updated DataFrame
test_feature

,ID,facilities,rating,location
0,0,gymrestaurantbarinternetswimmingpools,8.0 ExcellentFrom 1 reviews,Stokol
1,1,poolrestaurantgyminternetbar,7.4 Very GoodFrom 22 reviews,Hallerson
2,2,barswimmingpoolsinternetgym,0.0 FairFrom 4 reviews,Hallerson
3,3,gymswimmingpoolsbarintrnetrestaurant,6.8 Very GoodFrom 13 reviews,Andeman
4,4,gymrestaurantpoolbarintrnet,0.0 FairFrom 9 reviews,Hallerson
...,...,...,...,...
762,762,barpoolrestaurantgym,8.1 ExcellentFrom 9 reviews,Andeman
763,763,intrnetgympoolrestaurantbar,8.0 ExcellentFrom 4 reviews,Wanderland
764,764,restaurantgymintrnetbarswimmingpools,7.4 Very GoodFrom 19 reviews,Andeman
765,765,gyminternetbarpoolrestaurant,9.0 ExcellentFrom 17 reviews,Hallerson


Kita akan memisahkan harga dengan avg/night, dan mengambil angka tanpa tanda koma

In [ ]:
training_label

,Price
0,"13,500avg/night"
1,"13,000avg/night"
2,"19,000avg/night"
3,"6,000avg/night"
4,"20,000avg/night"
...,...
3061,"31,625avg/night"
3062,"30,500avg/night"
3063,"14,000avg/night"
3064,"8,500avg/night"


In [ ]:
# Extract the numeric value from the 'Price' column
training_label['Price'] = training_label['Price'].str.extract(r'(\d{1,3},\d{1,3})', expand=False)

# remove commas from the Price column
training_label['Price'] = training_label['Price'].str.replace(',', '')

# convert the Price column to integer
training_label['Price'] = training_label['Price'].astype(int)

# Print the updated DataFrame
training_label_new = training_label
training_label_new

,Price
0,13500
1,13000
2,19000
3,6000
4,20000
...,...
3061,31625
3062,30500
3063,14000
3064,8500


Akan menggabungkan antara dataset training_feature dan training_label

In [ ]:
# Select the desired columns from `training_features_cleaned`
df1 = training_features

# Select the desired columns from `data`
df2 = training_label_new

# Concatenate the two dataframes horizontally
training_features_1 = pd.concat([df1, df2], axis=1)

# Print the result
training_features_1

,facilities,rating,location,Price
0,restaurantbarswimmingpools,7.8 Very GoodFrom 10 reviews,Stokol,13500
1,intrnetrestaurantgym,5.6 GoodFrom 4 reviews,Machlessvile,13000
2,restaurantgympoolbar,7.2 Very GoodFrom 38 reviews,Wanderland,19000
3,barrestaurant,7.3 Very GoodFrom 6 reviews,Uberlandia,6000
4,internetrestaurant,7.2 Very GoodFrom 30 reviews,Stokol,20000
...,...,...,...,...
3061,barinternet,NaN,Andeman,31625
3062,restaurantbarinternet,8.1 ExcellentFrom 4 reviews,Uberlandia,30500
3063,barrestaurantswimmingpools,6.7 Very GoodFrom 10 reviews,Willsmian,14000
3064,restaurant,NaN,Hallerson,8500


In [ ]:
training_features_1.isna().sum().sort_values()

location        0
Price           0
facilities    301
rating        637
dtype: int64

In [ ]:
#training_features_1.to_csv('training_features_1.csv', header=True, index=False)
#files.download('training_features_1.csv')

# **DATA CLEANING**

Disini kami akan menghapus missing value yang bernilai NaN

In [ ]:
from operator import index
training_features_cleaned = training_features_1.dropna()
training_features_cleaned = training_features_cleaned.reset_index(drop=True)
training_features_cleaned['ID'] = training_features_cleaned.index
training_features_cleaned[['ID','facilities', 'rating', 'location', 'Price']]
df = training_features_cleaned[['facilities', 'rating', 'location', 'Price']]
df

,facilities,rating,location,Price
0,restaurantbarswimmingpools,7.8 Very GoodFrom 10 reviews,Stokol,13500
1,intrnetrestaurantgym,5.6 GoodFrom 4 reviews,Machlessvile,13000
2,restaurantgympoolbar,7.2 Very GoodFrom 38 reviews,Wanderland,19000
3,barrestaurant,7.3 Very GoodFrom 6 reviews,Uberlandia,6000
4,internetrestaurant,7.2 Very GoodFrom 30 reviews,Stokol,20000
...,...,...,...,...
2322,restaurantbar,7.6 Very GoodFrom 3 reviews,Andeman,5000
2323,barrestaurantpoolgym,7.8 Very GoodFrom 351 reviews,Andeman,30000
2324,restaurantbarinternet,8.1 ExcellentFrom 4 reviews,Uberlandia,30500
2325,barrestaurantswimmingpools,6.7 Very GoodFrom 10 reviews,Willsmian,14000


Dataset sudah bersih

# **MAKE A MODEL AND PREDICTION**

In [ ]:
!pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 483.6/483.6 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 18.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 5.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 5.5 MB/s eta 0:00:00

In [ ]:
!pip install --upgrade Pillow


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 36.8 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 8.4.0
    Uninstalling Pillow-8.4.0:
      Successfully uninstalled Pillow-8.4.0


In [ ]:
!pip uninstall -y pillow
!pip install pillow==7.2.0

Found existing installation: Pillow 9.5.0
Uninstalling Pillow-9.5.0:
  Successfully uninstalled Pillow-9.5.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.1/39.1 MB 27.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pillow: filename=Pillow-7.2.0-cp310-cp310-linux_x86_64.whl size=1305705 sha256=bab053386342b97f695ab66ba0d8089b33b66a33515989dfe9adf0b1a7f81fcf
  Stored in directory: /root/.cache/pip/wheels/fd/5c/a6/0ab3157e7e4a0ba139bd89cb1edbe0f87876cf55ef31b05d6a
Successfully built pillow
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imageio 2.25.1 requires pillow>=8.3.2, but you have pillow 7.2.0 which is incompatible.


In [ ]:
!pip install --upgrade packaging

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pycaret.regression import *
from pycaret.regression import setup

#Image.Resampling = None
s = setup(df, target='Price')


,Description,Value
0,Session id,7462
1,Target,Price
2,Target type,Regression
3,Original data shape,"(2327, 4)"
4,Transformed data shape,"(2327, 11)"
5,Transformed train set shape,"(1628, 11)"
6,Transformed test set shape,"(699, 11)"
7,Categorical features,3
8,Preprocess,True
9,Imputation type,simple


In [ ]:
best = compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
xgboost,Extreme Gradient Boosting,2379.1196,20485578.2949,4464.5098,0.9266,0.2692,0.1767,0.6300
rf,Random Forest Regressor,2469.4286,24173919.1544,4844.6218,0.9131,0.2799,0.1824,0.6500
gbr,Gradient Boosting Regressor,3169.7168,25179022.8489,4970.9835,0.9102,0.3170,0.2509,0.4270
et,Extra Trees Regressor,2835.4849,27241540.2283,5184.9242,0.9025,0.3168,0.2161,0.9920
dt,Decision Tree Regressor,2110.3889,29873893.8681,5375.3477,0.8923,0.3260,0.1555,0.4140
lightgbm,Light Gradient Boosting Machine,3326.4667,33895239.9397,5768.9755,0.8806,0.3143,0.2423,0.3220
ada,AdaBoost Regressor,4650.9726,38274577.6122,6164.8390,0.8624,0.4258,0.4095,0.3050
br,Bayesian Ridge,3788.9716,41738788.5344,6429.9406,0.8511,0.4610,0.3066,0.2100
en,Elastic Net,3790.1192,41777575.9333,6432.8756,0.8509,0.4541,0.3068,0.2140
llar,Lasso Least Angle Regression,3805.0961,41988313.1917,6449.1098,0.8501,0.4477,0.3079,0.4050


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

In [ ]:
print(best)

XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=-1, num_parallel_tree=None,
             predictor=None, random_state=7462, ...)


In [ ]:
finalize_model(best)

Pipeline(memory=FastMemory(location=/tmp/joblib),
         steps=[('numerical_imputer',
                 TransformerWrapper(include=[], transformer=SimpleImputer())),
                ('categorical_imputer',
                 TransformerWrapper(include=['facilities', 'rating',
                                             'location'],
                                    transformer=SimpleImputer(strategy='most_frequent'))),
                ('onehot_encoding',
                 TransformerWrapper(include=['location'],
                                    transformer=One...
                              feature_types=None, gamma=None, gpu_id=None,
                              grow_policy=None, importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, n_estimators=100,
                              n_jobs=-1, num_parallel_tree=None, predictor=None,
                              random_state=7462, ...))])

In [ ]:
evaluate_model(best)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [ ]:
predict_model(best)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Extreme Gradient Boosting,2271.3198,17146770.6488,4140.8659,0.9432,0.2770,0.1863


,facilities,rating,location,Price,prediction_label
346,restaurantbarintrnet,6.6 Very GoodFrom 4 reviews,Willsmian,7000,11114.878906
1074,barpoolrestaurant,5.2 GoodFrom 28 reviews,Uberlandia,40326,26578.980469
289,barrestaurantintrnet,8.1 ExcellentFrom 3 reviews,Willsmian,8000,7286.485840
2213,restaurant,6.5 Very GoodFrom 2 reviews,Stokol,5200,5872.332520
1449,internetrestaurantswimmingpoolsgymbar,7.2 Very GoodFrom 16 reviews,Ubisville,17000,17974.152344
...,...,...,...,...,...
1834,intrnetgymbarrestaurantswimmingpools,8.4 ExcellentFrom 226 reviews,Hallerson,43000,38007.027344
1593,restaurantbarinternet,8.0 ExcellentFrom 1 review,Hallerson,25000,20768.111328
1459,restaurantbarpool,7.9 Very GoodFrom 6 reviews,Stokol,9000,10406.086914
1542,restaurant,7.4 Very GoodFrom 12 reviews,Machlessvile,10000,10476.725586


In [ ]:
prediction_result = predict_model(best, test_feature)
prediction_result

,ID,facilities,rating,location,prediction_label
0,0,gymrestaurantbarinternetswimmingpools,8.0 ExcellentFrom 1 reviews,Stokol,19962.921875
1,1,poolrestaurantgyminternetbar,7.4 Very GoodFrom 22 reviews,Hallerson,19927.259766
2,2,barswimmingpoolsinternetgym,0.0 FairFrom 4 reviews,Hallerson,19927.259766
3,3,gymswimmingpoolsbarintrnetrestaurant,6.8 Very GoodFrom 13 reviews,Andeman,19639.966797
4,4,gymrestaurantpoolbarintrnet,0.0 FairFrom 9 reviews,Hallerson,19927.259766
...,...,...,...,...,...
762,762,barpoolrestaurantgym,8.1 ExcellentFrom 9 reviews,Andeman,19582.660156
763,763,intrnetgympoolrestaurantbar,8.0 ExcellentFrom 4 reviews,Wanderland,16912.349609
764,764,restaurantgymintrnetbarswimmingpools,7.4 Very GoodFrom 19 reviews,Andeman,19639.966797
765,765,gyminternetbarpoolrestaurant,9.0 ExcellentFrom 17 reviews,Hallerson,19927.259766


In [ ]:
prediction_result['Price'] = prediction_result['prediction_label']
prediction_result_1 = prediction_result[['ID','Price']]
prediction_result_1 = pd.DataFrame(prediction_result[['ID','Price']])
prediction_result_1

,ID,Price
0,0,19962.921875
1,1,19927.259766
2,2,19927.259766
3,3,19639.966797
4,4,19927.259766
...,...,...
762,762,19582.660156
763,763,16912.349609
764,764,19639.966797
765,765,19927.259766


In [ ]:
!ls

catboost_info  drive  logs.log	One-Step-Ahead-result.csv  sample_data


In [ ]:
from google.colab import files
prediction_result_1.to_csv('One-Step-Ahead-result.csv', header=True, index=False)

In [ ]:
files.download('One-Step-Ahead-result.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!pwd

/content
